In [ ]:
# -*- coding: utf-8 -*-
"""ORAN-hua.ipynb

In [ ]:
Automatically generated by Colab.

In [ ]:
Original file is located at
    https://colab.research.google.com/drive/1Qq02OhKqSD6YaaiiW-g1Su5OhXMuxPGO
"""

comment out this if you run on google colab

In [ ]:
import matplotlib

In [ ]:
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt

In [ ]:
import csv
import random
import math

In [ ]:
seed = 2024
random.seed(seed)
# Function to generate random throughput based on slice value
# def generate_throughput(slice_value):
#     if slice_value == 1:
#         return random.randint(85, 200)
#     elif slice_value == 2:
#         return random.randint(30, 300)
#     elif slice_value == 3:
#         return random.randint(1500, 90000)
#     elif slice_value == 4:
#         return random.randint(900, 8000)
#     else:
#         return None
#
# # Open CSV file for writing
# with open('data/data.csv', mode='w', newline='') as file:
#     writer = csv.writer(file)
#     writer.writerow(['id', 'slice', 'x', 'y', 'throughput(Kbps)', 'allocatedCell'])
#
#     # Generate 250 rows of data
#     for _ in range(250):
#         # random.seed(2024)
#         slice_value = random.randint(1, 4)
#         x = random.randint(0, 99)
#         y = random.randint(0, 99)
#         throughput = generate_throughput(slice_value)
#         writer.writerow([(_+1), slice_value, x, y, throughput, 0])

In [ ]:
import pandas as pd

In [ ]:
ues = pd.read_csv('data/data.csv')


**Our Policy**
<br>
import json<br>
policy = {<br>
    '1': {<br>
        # Preference 1: Non-anchored Macro cell, since we only need GBR and there are no latency or delay requirements<br>
        'p1': 'macro',<br>
        # Preference 2: Non-anchored Pico cell, if there's no available macro cell coverage, we can use the nearest pico cell<br>
        'p2': 'pico',<br>
        'p3': 'anchored'<br>
    },<br>
    '2': {<br>
        # Preference 1: Non-anchored macro cell, since the utility is just to sync up the IoT device or an activity not requiring high throughputs<br>
        'p1': 'macro',<br>
        # Preference 2: Because of increased ranges of the anchored cell, to keep the GBR up if there's an emergency signal needed to be sent and pico is too far<br>
        'p2': 'anchored',<br>
        'p3': 'pico'<br>
    },<br>
    '3': {<br>
        # Preference 1: For an emBB use-case, we will need the highest available throughput<br>
        'p1': 'pico',<br>
        # Preference 2: Anchored cell since it would ensure lesser bursts/buffers in the connection<br>
        'p2': 'anchored',<br>
        'p3': 'macro'<br>
    },<br>
    '4': {<br>
        # Preference 1: To minimize the latency, anchored connections would be given the first ask<br>
        'p1': 'anchored',<br>
        # Preference 2: To mitigate the latency delays, macro cells preferred because of their increased ranges as when compared against pico cells<br>
        'p2': 'macro',<br>
        'p3': 'pico'<br>
    }<br>
}<br>
efine the environment


In [ ]:
import numpy as np
import gym
from gym import spaces

In [ ]:
class OranEnv(gym.Env):
    """
    Custom Environment that follows gym interface.
    This is a simple env.
    """
    # Because of google colab, we cannot implement the GUI ('human' render mode)
    metadata = {'render.modes': ['console']}
    # Define constants for clearer code
    ues = pd.read_csv('data/data.csv')
    seed = 2024
    def __init__(self, policy, cell_file='data/cells.csv'):
        super(OranEnv, self).__init__()

        # Firsts: UEs which were allocated their first preference
        # Allocated: the actual allocated cell IDs
        # oloads: Number of times the allocation WOULD HAVE lead to an overload
        # unmanaged: Number of UEs which could not be allocated any cell
        self.firsts = []
        self.allocated = []
        self.oloads = []
        self.unmanaged = 0
        self.timestep = 0
        self.cells = pd.read_csv(cell_file)
        self.availableCells=[]
        self.policy = policy

        # Define circular areas
        self.circle_centers = np.array(self.cells[['x','y']].values).tolist()
        self.circle_radii = self.cells['radius'].to_list()

        # for i in range(0, len(cells['x'])):
        #     self.circle_centers.append((cells['x'][i], cells['y'][i]))
        #     self.circle_radii.append(cells['radius'][i])

In [ ]:
        # Define action and observation space
        # They must be gym.spaces objects
        # Example when using discrete actions, we have four in each policy
        n_actions = 4
        self.action_space = spaces.Discrete(n_actions)
        # The observation will be the coordinate of the agent
        # this can be described both by Discrete and Box space
        self.observation_space = spaces.Box(low=0, high=self.cells.shape[0],
                                            shape=(ues.shape[0],), dtype=np.float32)
    def reset(self):
        """
        Important: the observation must be a numpy array
        :return: (np.array)
        """
        self.firsts = []
        self.allocated = []
        self.oloads = []
        self.unmanaged = 0
        self.timestep = 0
        self.availableCells=[]

        # here we convert to float32 to make it more general (in case we want to use continuous actions)
        return np.array([0]*ues.shape[0]).astype(np.float32)
    def step(self, action):
        pref = self.policy[str(action)]

        # Account for the boundaries of the grid
        ue = ues.loc[self.timestep] # originally it is i - the index of ue
        res1 = self.findCell(int(ue['id']), pref['p1'], ue['x'], ue['y'], ue['throughput(Kbps)'], True)
        if res1 != False:
            # print("P1")
            ue['allocatedCell'] = res1
            ues.loc[self.timestep] = ue
        else:
            # print("P2")
            res2 = self.findCell(ue['id'], pref['p2'], ue['x'], ue['y'], ue['throughput(Kbps)'], False)
            if res2 != False:
                ue['allocatedCell'] = res2
                ues.iloc[self.timestep] = ue
            else:
                # print("P3")
                res3 = self.findCell(ue['id'], pref['p3'], ue['x'], ue['y'], ue['throughput(Kbps)'], False)
                if res3 != False:
                    ue['allocatedCell'] = res3
                    ues.iloc[self.timestep] = ue
                else:
                    self.unmanaged += 1
                    # print('Sorry, no cell for this UE.')

        # Are we at the left of the grid?
        done = bool(self.timestep == (ues.shape[0]-1))

        # Null reward everywhere except when reaching the goal (left of the grid)
        reward = self.get_reward()

        # Optionally we can pass additional info, we are not using that for now
        info = {"current_index": self.timestep}
        self.timestep += 1
        return np.array(ues['allocatedCell']).astype(np.float32), reward, done, info
    def render(self, mode='console'):
        if mode != 'console':
            raise NotImplementedError()

        # Define circular areas
        self.circle_centers = np.array(self.cells[['x', 'y']].values).tolist()
        self.circle_radii = self.cells['radius'].to_list()

        # Plotting
        plt.figure(figsize=(10, 10))

        # Plot circular areas and add label numbers
        for i, (center, radius) in enumerate(zip(self.circle_centers, self.circle_radii)):
            if radius == 6:
                circle = plt.Circle(center, radius, alpha=0.3, color='red')
            elif radius == 30:
                circle = plt.Circle(center, radius, alpha=0.3, color='blue')
            else:
                circle = plt.Circle(center, radius, alpha=0.3, color='lightblue')
            plt.gca().add_patch(circle)

            # Add label number inside the circle
            plt.text(center[0], center[1], str(i + 1), color='black', ha='center', va='center')

        # Adjust plot settings
        plt.title('Arrangement of the cells in a 100 km x 100 km space')
        plt.axis([0, 100, 0, 100])

        # Code to see the UE points scattered on the plane
        a = ues['x'].to_list()
        b = ues['y'].to_list()
        plt.scatter(a, b)

        # Show the plot
        plt.show()
    def close(self):
        pass

In [ ]:
    def seed(self, seed: int | None = None):
        """Set the seed for this env's random number generator(s)."""
        pass
    def get_reward(self):
        reward = 0
        if self.timestep == ues.shape[0]-1:
            # only return at the end of the episode
            for i in range(0, len(ues)):
                self.allocated.append({'ueId': int(ues.iloc[i]['id']), 'allocatedCell': int(ues.iloc[i]['allocatedCell'])})
            print("Number of UEs managed: ", len(self.allocated))
            percentageFirst = (len(self.firsts) / len(self.allocated)) * 100
            print("Percentage of UEs allocated their first preferences: ", percentageFirst)
            print("Overloads: ", len(self.oloads))
            print("Unmanaged: ", self.unmanaged)
            reward = percentageFirst
        return reward

    # Function to calculate Euclidean distance between a UE and a cell
    @staticmethod
    def dist(uex, uey, cellx, celly):
        return math.sqrt((uex - cellx) ** 2 + (uey - celly) ** 2)
    def findCell(self, id, preference, x, y, throughput, first):
        if preference == 'macro':
            self.availableCells = self.cells[self.cells.pico == 0]
            self.availableCells = self.availableCells[self.availableCells.anchored == 0]
        if preference == 'pico':
            self.availableCells = self.cells[self.cells.pico == 1]
            self.availableCells = self.availableCells[self.availableCells.anchored == 0]
        if preference == 'anchored':
            self.availableCells = self.cells[self.cells.anchored == 1]
        distances = []
        for i in range(0, len(self.availableCells)):
            distances.append(self.dist(x, y, self.availableCells.iloc[i]['x'], self.availableCells.iloc[i]['y']))
        self.availableCells.insert(len(self.availableCells.columns), 'distance', distances)
        self.availableCells = self.availableCells.sort_values('distance')
        # print("UE ID: ",id, "Preferred Cell: ", preference)
        # print(availableCells)
        for i in range(0, len(self.availableCells)):
            ac = self.availableCells.iloc[i]
            cell = self.cells.loc[self.cells['id'] == ac['id']]
            # print(cell)
            x = int(cell['currCapacity'])
            # print(x-throughput)
            if ((x - throughput) < 0):
                self.oloads.append(1)
                # print("Overloading")
            else:
                cell['currCapacity'] = cell['currCapacity'] - throughput
                self.cells.loc[self.cells['id'] == ac['id']] = cell
                # ues.iloc[id-1]['allocatedCell'] = minId
                if (first == True):
                    self.firsts.append({'ueId': int(id), 'allocatedCell': int(cell['id'])})
                return ac['id']
            #   print("Available")
        return False

from stable_baselines3 import DQN, PPO2, A2C, ACKTR

In [ ]:
from stable_baselines3.common.env_util import make_vec_env

In [ ]:
policy = {
    '1': {
        # Preference 1: Non-anchored Macro cell, since we only need GBR and there are no latency or delay requirements
        'p1': 'macro',
        # Preference 2: Non-anchored Pico cell, if there's no available macro cell coverage, we can use the nearest pico cell
        'p2': 'pico',
        'p3': 'anchored'
    },
    '2': {
        # Preference 1: Non-anchored macro cell, since the utility is just to sync up the IoT device or an activity not requiring high throughputs
        'p1': 'macro',
        # Preference 2: Because of increased ranges of the anchored cell, to keep the GBR up if there's an emergency signal needed to be sent and pico is too far
        'p2': 'anchored',
        'p3': 'pico'
    },
    '3': {
        # Preference 1: For an emBB use-case, we will need the highest available throughput
        'p1': 'pico',
        # Preference 2: Anchored cell since it would ensure lesser bursts/buffers in the connection
        'p2': 'anchored',
        'p3': 'macro'
    },
    '4': {
        # Preference 1: To minimize the latency, anchored connections would be given the first ask
        'p1': 'anchored',
        # Preference 2: To mitigate the latency delays, macro cells preferred because of their increased ranges as when compared against pico cells
        'p2': 'macro',
        'p3': 'pico'
    }
}

Instantiate the env

In [ ]:
env = OranEnv(cell_file='data/cells.csv', policy=policy)
# wrap it
env = make_vec_env(lambda: env, n_envs=1)

Train the agent<br>
model = ACKTR('MlpPolicy', env, verbose=1).learn(5000)

Test the trained agent

In [ ]:
obs = env.reset()
n_steps = len(ues)
for step in range(n_steps):
  # action, _ = model.predict(obs, deterministic=True)
  action = [ues.loc[step]['slice']]
  print("Step {}".format(step + 1))
  print("Action: ", action)
  obs, reward, done, info = env.step(action)
  print('obs=', obs, 'reward=', reward, 'done=', done)

In [ ]:
  if done:
    # Note that the VecEnv resets automatically
    # when a done signal is encountered
    env.render(mode='console')
    print("Goal reached!", "reward=", reward)
    break


**Baseline Policy**
<br>
import json<br>
policy2 = {<br>
    '1': {<br>
        # Preference 1: Non-anchored Macro cell, since we only need GBR and there are no latency or delay requirements<br>
        'p1': 'macro',<br>
        # Preference 2: Non-anchored Pico cell, if there's no available macro cell coverage, we can use the nearest pico cell<br>
        'p2': 'pico',<br>
        # 'p3': 'anchored'<br>
    },<br>
    '2': {<br>
        # Preference 1: Non-anchored macro cell, since the utility is just to sync up the IoT device or an activity not requiring high throughputs<br>
        'p1': 'macro',<br>
        # Preference 2: Because of increased ranges of the anchored cell, to keep the GBR up if there's an emergency signal needed to be sent and pico is too far<br>
        # 'p2': 'anchored',<br>
        'p2': 'pico'<br>
    },<br>
    '3': {<br>
        # Preference 1: For an emBB use-case, we will need the highest available throughput<br>
        'p1': 'pico',<br>
        # Preference 2: Anchored cell since it would ensure lesser bursts/buffers in the connection<br>
        # 'p2': 'anchored',<br>
        'p2': 'macro'<br>
    },<br>
    '4': {<br>
        # Preference 1: To minimize the latency, anchored connections would be given the first ask<br>
        # 'p1': 'anchored',<br>
        # Preference 2: To mitigate the latency delays, macro cells preferred because of their increased ranges as when compared against pico cells<br>
        'p1': 'macro',<br>
        'p2': 'pico'<br>
    }<br>
}<br>
# RUNNING THIS BLOCK AGAIN FOR THE BASELINE POLICY PERFORMANCE<br>
policy = {<br>
    '1': {<br>
        # Preference 1: Non-anchored Macro cell, since we only need GBR and there are no latency or delay requirements<br>
        'p1': 'macro',<br>
        # Preference 2: Non-anchored Pico cell, if there's no available macro cell coverage, we can use the nearest pico cell<br>
        'p2': 'pico',<br>
        'p3': 'anchored'<br>
    },<br>
    '2': {<br>
        # Preference 1: Non-anchored macro cell, since the utility is just to sync up the IoT device or an activity not requiring high throughputs<br>
        'p1': 'macro',<br>
        # Preference 2: Because of increased ranges of the anchored cell, to keep the GBR up if there's an emergency signal needed to be sent and pico is too far<br>
        'p2': 'anchored',<br>
        'p3': 'pico'<br>
    },<br>
    '3': {<br>
        # Preference 1: For an emBB use-case, we will need the highest available throughput<br>
        'p1': 'pico',<br>
        # Preference 2: Anchored cell since it would ensure lesser bursts/buffers in the connection<br>
        'p2': 'anchored',<br>
        'p3': 'macro'<br>
    },<br>
    '4': {<br>
        # Preference 1: To minimize the latency, anchored connections would be given the first ask<br>
        'p1': 'anchored',<br>
        # Preference 2: To mitigate the latency delays, macro cells preferred because of their increased ranges as when compared against pico cells<br>
        'p2': 'macro',<br>
        'p3': 'pico'<br>
    }<br>
}<br>
# Instantiate the env<br>
env = OranEnv(cell_file='data/cells2.csv', policy=policy2)<br>
# wrap it<br>
env = make_vec_env(lambda: env, n_envs=1)<br>
# Test the trained agent<br>
obs = env.reset()<br>
n_steps = len(ues)<br>
for step in range(n_steps):<br>
  # action, _ = model.predict(obs, deterministic=True)<br>
  action = [ues.loc[step]['slice']]<br>
  print("Step {}".format(step + 1))<br>
  print("Action: ", action)<br>
  obs, reward, done, info = env.step(action)<br>
  print('obs=', obs, 'reward=', reward, 'done=', done)<br>
  if done:<br>
    # Note that the VecEnv resets automatically<br>
    # when a done signal is encountered<br>
    env.render(mode='console')<br>
    print("Goal reached!", "reward=", reward)<br>
    break